# SIT320 Advanced Algorithms
## Module 12 - MDP and Reinforcement Learning

---

<img src="uml-class-diagram.png" width="1000" height="800">

---

### Board Class

In [ ]:
from abc import ABC, abstractmethod

class Board(ABC):
    def __init__(self, boardDimensions):
        self.boardDimensions = boardDimensions

    @abstractmethod
    def createBoard(self):
        pass

    @abstractmethod
    def printBoard(self):
        pass

    @abstractmethod
    def setCellState(self, position, state):
        pass

    @abstractmethod
    def getCellState(self, position):
        pass

    @abstractmethod
    def getBoardState(self):
        pass

    @abstractmethod
    def getBoardDimensions(self):
        pass

    @abstractmethod
    def spaceIsFree(self, position):
        pass

    

In [ ]:
class TicTacToeBoard(Board):
    def __init__(self, boardDimensions):
        super().__init__(boardDimensions)
        self.createBoard()
        """Create a board of dimensions boardDimensions x boardDimensions
        Args: boardDimensions (int): the dimensions of the board
        """


    def createBoard(self):
        """Create a board of dimensions boardDimensions x boardDimensions"""
        self.boardState = {i+1: ' ' for i in range(self.getBoardDimensions()**2)}

    def printBoard(self):
        for i in range(self.boardDimensions):
            row = [self.boardState[i*self.boardDimensions+j+1] for j in range(self.boardDimensions)]
            print('|'.join(row))
            if i < self.boardDimensions-1:
                print('-'*(self.boardDimensions*2-1))
        print('\n')

    def setCellState(self, position, state):
        """Set the state of a cell on the board
        Args:
            position (int): the position of the cell
            state (str): the state of the cell
        """
        self.boardState[position] = state

    def getCellState(self, position):
        """Get the state of a cell on the board
        Args:
            position (int): the position of the cell
        Returns: the state of the cell
        """
        return self.boardState[position]

    def getBoardState(self):
        """Get the state of the board
        Returns: A dictionary with keys 1 to boardDimensions**2 and values 'X', 'O' or ' '
        """
        return self.boardState

    def getBoardDimensions(self):
        """Get the dimensions of the board
        Returns: An integer representing the dimensions of the board 
        """
        return self.boardDimensions
    
    def spaceIsFree(self, position):
        if self.boardState[position] == ' ':
            return True

---

### Game Logic

In [ ]:
from abc import ABC, abstractmethod

class GameLogic():
    def __init__(self, boardGame):
        self.boardGame = boardGame

    @abstractmethod
    def chkForkWin(self):
        pass

    @abstractmethod
    def chkForDraw(self):
        pass

    @abstractmethod
    def chkMarkForWin(self, letter):
        pass

In [ ]:
class TicTacToeGameLogic(GameLogic):
    def __init__(self, boardGame):
        super().__init__(boardGame)
        """Create a game logic object for the board game
        Args: boardGame (Board): the board game. Must be a subclass of Board
        """

    def chkForDraw(self):
        """Check if the game is a draw.
        Returns: bool: True if the game is a draw, False otherwise.
        """
        boardState = self.boardGame.getBoardState()
        return all(boardState[key] != ' ' for key in boardState.keys())

    def chkForWin(self):
        """Check if any player has won.
        Returns: bool: True if any player has won, False otherwise.
        """
        boardState = self.boardGame.getBoardState()
        boardDimensions = self.boardGame.getBoardDimensions()
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] != ' ':
                return True
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] != ' ':
                return True
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] != ' ':
            return True
        return len(set(diagonal2)) == 1 and diagonal2[0] != ' '

    def chkMarkForWin(self, letter):
        """Check if the player with the specified letter has won.
        Args: letter (str): Letter of the player to check for win.
        Returns: bool: True if the player with the specified letter has won, False otherwise.
        """
        boardState = self.boardGame.getBoardState()
        boardDimensions = self.boardGame.getBoardDimensions()
        # check rows
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] == letter:
                return True
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] == letter:
                return True
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] == letter:
            return True
        return len(set(diagonal2)) == 1 and diagonal2[0] == letter

---

### Player Class

In [ ]:
from abc import ABC, abstractmethod

class Player(ABC):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
        """Create a player object
        Args: letter (str): the letter of the player. Must be 'X' or 'O'
        Args: algorithm (Algorithm): the algorithm used by the player. Must be a subclass of Algorithm
        """


    @abstractmethod
    def makeMove(self, boardGame):
        pass

In [ ]:
class HumanPlayer(Player):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
    
    # function for player to choose a position
    def makeMove(self, boardGame):
        """Make a move by asking for input from the user.
        Args: boardGame (Board): The board game object.
        If the position is not free, ask for another position.
        If the position is free, set the cell state to the player's letter.
        """
        position = self.algorithm.bestMove(boardGame, self.letter)
        boardGame.setCellState(position, self.letter)

In [ ]:
class ComputerPlayer(Player):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
    
    def makeMove(self, boardGame):
        """Make a move by using the algorithm to find the best move.
        Args: boardGame (Board): The board game object.
        Set the cell state to the player's letter.
        """
        position = self.algorithm.bestMove(boardGame, self.letter)
        boardGame.setCellState(position, self.letter)

---

### Algorithms

In [ ]:
class Algorithm(ABC):
    def __init__(self, boardGame):
        self.boardGame = boardGame
        """Create an algorithm object for the board game
        Args: boardGame (Board): the board game. Must be a subclass of Board
        """
    
    @abstractmethod
    def bestMove(self, boardGame, letter):
        pass


### MiniMax

In [ ]:
class Minimax(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        """Find the best move for the computer player.
        Args: boardGame (Board): The board game object
        Args: letter (str): The letter of the computer player.
        Returns: int: The position of the best move."""
        boardState = boardGame.getBoardState()
        bestScore = -1000
        bestMove = 0
        for key in boardState.keys():
            if boardState[key] == ' ':
                boardState[key] = letter
                score = self.minimax(boardState, 0, False, letter)
                boardState[key] = ' '
                if score > bestScore:
                    bestScore = score
                    bestMove = key
        return bestMove

    def minimax(self, boardState, depth, isMaximizing, letter, maxDepth=5):
        """Find the best score for the computer player.
        Args: boardState (dict): The board state.
        Args: depth (int): The depth of the tree.
        Args: isMaximizing (bool): Whether the player is maximizing or not.
        Args: letter (str): The letter of the computer player.
        Args: maxDepth (int): The maximum depth of the tree.
        Returns: int: The best score for the computer player."""
        
        gameLogic = TicTacToeGameLogic(self.boardGame)
        opponentLetter = 'O' if letter == 'X' else 'X'
        if(gameLogic.chkMarkForWin(letter)):
            return 1
        elif(gameLogic.chkMarkForWin(opponentLetter)):
            return -1
        elif(gameLogic.chkForDraw()):
            return 0
        elif depth >= maxDepth:
            return 0

        if isMaximizing:
            bestScore = -1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = letter
                    score = self.minimax(boardState, depth + 1, False, letter)
                    boardState[key] = ' '
                    if score > bestScore:
                        bestScore = score
        else:
            bestScore = 1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = opponentLetter
                    score = self.minimax(boardState, depth + 1, True, letter)
                    boardState[key] = ' '
                    if score < bestScore:
                        bestScore = score
        return bestScore

### Minimax with Alpha-Beta Pruning

In [ ]:
class MinimaxAlphaBeta(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        boardState = boardGame.getBoardState()
        bestScore = -1000
        bestMove = 0
        for key in boardState.keys():
            if boardState[key] == ' ':
                boardState[key] = letter
                score = self.minimax(boardState, 0, False, letter)
                boardState[key] = ' '
                if score > bestScore:
                    bestScore = score
                    bestMove = key
        return bestMove

    def minimax(self, boardState, depth, isMaximizing, letter, alpha=-1000, beta=1000, maxDepth=5):
        """Find the best score for the computer player.
        Args: boardState (dict): The board state.
        Args: depth (int): The depth of the tree.
        Args: isMaximizing (bool): Whether the player is maximizing or not.
        Args: letter (str): The letter of the computer player.
        Args: alpha (int): The alpha value. 
        Args: beta (int): The beta value.
        Args: maxDepth (int): The maximum depth of the tree.
        Returns: int: The best score for the computer player."""
        gameLogic = TicTacToeGameLogic(self.boardGame)
        opponentLetter = 'O' if letter == 'X' else 'X'
        if(gameLogic.chkMarkForWin(letter)):
            return 1
        elif(gameLogic.chkMarkForWin(opponentLetter)):
            return -1
        elif(gameLogic.chkForDraw()):
            return 0
        elif depth >= maxDepth:
            return 0

        if isMaximizing:
            bestScore = -1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = letter
                    score = self.minimax(boardState, depth+1, False, letter, alpha, beta)
                    boardState[key] = ' '
                    bestScore = max(score, bestScore)
                    alpha = max(alpha, score)
                    if beta <= alpha:
                        break
        else:
            bestScore = 1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = opponentLetter
                    score = self.minimax(boardState, depth+1, True, letter, alpha, beta)
                    boardState[key] = ' '
                    bestScore = min(score, bestScore)
                    beta = min(beta, score)
                    if beta <= alpha:
                        break
        return bestScore

## Reinforcement Learning 

---


### Value Iteration
- These classes solve the tic-tac-toe game using the value iteration algorithm.
- We assume the tic-tac-toe game is an MDP with states, actions, transition probabilities, and rewards.
- States: the board state.
- Actions: the position to place the next move.
- Transition probabilities: 1.0 if the move is valid, 0.0 otherwise.
- Rewards: 1 if the move is a winning move, 0 otherwise.

In [ ]:
import random

class ValueIteration(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)
        policy =  {}
        G = Graph()
        self.valueIteration()


    def bestMove(self, boardGame, letter):
        # select best move from policy
        boardState = boardGame.getBoardState()
        return self.policy[boardState]

    def valueIteration(self):
        best_policy = {}
        value_function = {}
        gameLogic = TicTacToeGameLogic(self.boardGame)
        self.intialize()
        self.converge()
        self.policy = best_policy

    def initialize(self, graph, gameLogic):
        """ Initialize the value function and policy
            Args: graph (Graph): The graph object.
            Create all the states and edges in the graph.
        """
        # create all the states
        for state in self.board_game.get_all_states(self.board_game.getBoardDimensions(), gameLogic):
            graph.add_node(state)
        # create all the edges
        for state in graph.nodes.keys():
            for action in self.board_game.get_actions(state):
                new_state = self.board_game.get_new_state(state, action)
                graph.add_edge(state, new_state)
        # initialize value function and policy
        for state in graph.nodes.keys():
            self.value_function[state] = 0  # Initialize to 0
            self.policy[state] = None  # Initialize to None
    
    def converge(self, graph, gamma=0.9, theta=1e-4):
        """ Input: graph (Graph): The graph object.
                   gamma (float): The discount factor.
                   theta (float): The convergence threshold.
            Output: value_function (dict): The value function.
                    policy (dict): The policy.
            Iterate over all states and update the value function and policy until convergence.
        """
        new_value = 0  # Calculate new value based on Bellman equation
        while True:
            delta = 0
            for state in graph.nodes.keys():
                old_value = self.value_function[state]
                # Update policy here based on new_value
                new_value = self.get_value(state)
                self.value_function[state] = new_value
                delta = max(delta, abs(old_value - new_value))
            self.extract_policy(graph)
            if delta < theta:
                break

    def extract_policy(self, graph):
        """ Input: graph (Graph): The graph object.
            Output: policy (dict): The policy.
            Iterate over all states and update the policy based on the value function.
        """
        for state in graph.nodes.keys():
            actions = self.board_game.get_actions(state)
            # Update policy here based on new_value
            self.policy[state] = max(actions, key=lambda action: self.get_value(state, action))
        return self.policy


    def get_value(self, state, action=None):
        """ Input: state (tuple): The state.
                   action (tuple): The action.
            Output: value (float): The value.
        """
        # Calculate the value of a state by taking the maximum value of the Bellman equation over all possible actions.
        if action is None:
            return max(self.bellman_equation(state, a) for a in self.board_game.get_actions(state))
        else:
            return self.bellman_equation(state, action)

    def bellman_equation(self, state, action):
        """ Input: state (tuple): The state.
                   action (tuple): The action.
            Output: value (float): The value.
            Uses the Bellman equation to calculate the value of a state after taking a given action.
        """
    # Calculate the value of a state after taking a given action, using the Bellman equation.
        reward = self.get_reward(state, action)
        gamma = self.get_discount_factor(state)
        transition_probabilities = self.get_transition_probabilities(state, action)
        value_function = sum(transition_probabilities[new_state] * self.value_function[new_state] for new_state in transition_probabilities.keys())
        return reward + gamma * value_function
    
    def get_reward(self, state, action):
        """ Input: state (tuple): The state.
                   action (tuple): The action.
            Output: reward (float): The reward.
            Calculate the reward for a given state and action.
        """
        return 1 if self.board_game.chkMarkForWin(state, action) else 0


    def get_transition_probabilities(self, actions):
        """ Input: state (tuple): The state.
                   action (tuple): The action.
            Output: transition_probabilities (dict): The transition probabilities.
            Calculate the transition probabilities for a given state and action.
        """
        probs = [random.random() for _ in actions]
        total = sum(probs)
        return {action: prob / total for action, prob in zip(actions, probs)}
    

In [ ]:
class Node:
    def __init__(self, state):
        self.state = state
        self.value = 0  # Initialize value function for this state
        

class Graph:
    def __init__(self):
        self.nodes = {}
    
    def add_node(self, state):
        node = Node(state)
        self.nodes[state] = node
        return node

    def add_edge(self, state1, state2):
        node1 = self.nodes.get(state1) or self.add_node(state1)
        node2 = self.nodes.get(state2) or self.add_node(state2)



### Q-Learning

In [ ]:
class QLearning(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)
    
    def bestMove(self, boardGame, letter):
        pass

---

### Algorithm for the human player

In [ ]:
class UserInput(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        """Ask the user for input.
        Args: boardGame (Board): The board game object.
        Args: letter (str): The letter of the computer player.
        Returns: int: The position of the user's input."""
        while True:
            try:
                position = int(input("Please enter a position: "))
                if position < 1 or position > boardGame.getBoardDimensions()**2:
                    raise ValueError
                if boardGame.spaceIsFree(position):
                    return position
                else:
                    raise ValueError
            except ValueError:
                print("Invalid input!")

In [ ]:
class AbstractGameFactory(ABC):
    @abstractmethod
    def createBoard(self) -> Board:
        pass

    @abstractmethod
    def createGameLogic(self) -> GameLogic:
        pass

    @abstractmethod
    def createPlayer(self) -> Player:
        pass

    @abstractmethod
    def createAlgorithm(self) -> Algorithm:
        pass

In [ ]:
class TicTacToeCreator(AbstractGameFactory):
    def __init__(self, dimensions):
        self.board = TicTacToeBoard(dimensions)

    def createBoard(self) -> Board:
        """Create a board of dimensions boardDimensions x boardDimensions
        Returns: board (Board): the board game. Must be a subclass of Board
        """
        return self.board

    def createGameLogic(self) -> GameLogic:
        """Create a game logic object for the board game
        Returns: gameLogic (GameLogic): the game logic object. Must be a subclass of GameLogic
        """
        return TicTacToeGameLogic(self.board)

    def createPlayer(self, letter, isComputer, algorithm) -> Player:
        """Create a player object
        Args: letter (str): the letter of the player. Must be 'X' or 'O'
        Args: isComputer (bool): whether the player is a computer or not
        Args: algorithm (Algorithm): the algorithm used by the player. Must be a subclass of Algorithm
        Returns: player (Player): the player object. Must be a subclass of Player
        """
        if isComputer:
            return ComputerPlayer(letter, algorithm)
        return HumanPlayer(letter, algorithm)

    def createAlgorithm(self, algorithm) -> Algorithm:
        """Create an algorithm object for the board game
        Args: algorithm (int): the algorithm used by the player. Must be 1, 2, 3 or 4
        Returns: algorithm (Algorithm): the algorithm object. Must be a subclass of Algorithm
        """
        if algorithm == 1:
            return Minimax(self.board)
        elif algorithm == 2:
            return MinimaxAlphaBeta(self.board)
        elif algorithm == 3:
            return ValueIteration(self.board)
        elif algorithm == 4:
            return QLearning(self.board)
        elif algorithm == 5:
            return UserInput(self.board)

---

## Game

In [ ]:
import time

def prompt_user(prompt, options):
    """Prompt the user to choose an option.
    Args: prompt (str): The prompt to display to the user.
    Args: options (list): The list of options to display to the user.
    Returns: int: The option chosen by the user.
    """
    print(prompt)
    for i, option in enumerate(options):
        print(f"{i+1}. {option}")
    choice = int(input("Please choose an option: "))
    while choice < 1 or choice > len(options):
        print(f"Choice must be between 1 and {len(options)}!")
        choice = int(input("Please choose an option: "))
    return choice

def choose_game():
    """Prompt the user to choose a game.
    Returns: game_factory (AbstractGameFactory): The game factory for the game chosen by the user.
    """
    prompt = "Welcome to the Game Factory!\nPlease choose a game:"
    options = ["Tic Tac Toe", "Chess", "Backgammon"]
    choice = prompt_user(prompt, options)
    if choice == 1:
        dimensions = int(input("Please enter the board dimensions: "))
        return TicTacToeCreator(dimensions)
    elif choice == 2:
        return ChessCreator()
    elif choice == 3:
        return BackgammonCreator()

def choose_algorithm():
    """Prompt the user to choose an algorithm.
    Returns: int: The algorithm chosen by the user.
    """
    prompt = "Which Algorithm should player use?"
    options = ["Minimax", "Minimax with Alpha Beta Pruning", "Value Iteration", "Q-Learning", "User Input"]
    return prompt_user(prompt, options)

def create_player(game_factory, letter):
    """Create a player object
    Args: game_factory (AbstractGameFactory): The game factory for the game chosen by the user.
    Args: letter (str): the letter of the player. Must be 'X' or 'O'
    Returns: player (Player): the player object. Must be a subclass of Player
    """
    is_computer = input(f"Is player {letter} a computer? (Y/N): ")
    while is_computer not in ['Y', 'N']:
        print("Invalid input!")
        is_computer = input(f"Is player {letter} a computer? (Y/N): ")
    algorithm_choice = choose_algorithm()
    algorithm = game_factory.createAlgorithm(algorithm_choice)
    return game_factory.createPlayer(letter, is_computer == 'Y', algorithm)

def play_game():
    """Play the game."""
    game_factory = choose_game()
    game_logic = game_factory.createGameLogic()
    player_one = create_player(game_factory, 'X')
    player_two = create_player(game_factory, 'O')
    game_factory.board.printBoard()
    start = time.time()
    while not game_logic.chkForWin() and not game_logic.chkForDraw():
        player_one.makeMove(game_factory.board)
        game_factory.board.printBoard()
        if game_logic.chkForWin():
            print("Player", player_one.letter, "wins!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        elif game_logic.chkForDraw():
            print("It's a draw!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        player_two.makeMove(game_factory.board)
        game_factory.board.printBoard()
        if game_logic.chkForWin():
            print("Player", player_two.letter, "wins!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        elif game_logic.chkForDraw():
            print("It's a draw!")
            end = time.time()
            print("Time taken: ", end - start)
            break

play_game()